In [1]:
using Laplacians

In [2]:
pwd()

"/Users/serbanstan/git/Laplacians.jl/debug/Solver Testing"

In [3]:
include("../../src/primalDualIPM.jl");

In [4]:
solvers = [augTreeSolver, KMPSDDSolver, samplingSDDSolver, AMGSolver]

4-element Array{Function,1}:
 Laplacians.augTreeSolver    
 Laplacians.KMPSDDSolver     
 Laplacians.samplingSDDSolver
 Laplacians.AMGSolver        

In [5]:
n = 10000

nlist = []
ilist = []
edges = []
targetTol = []

iters = [Array{Float64,1}(0) for s in solvers]
precision = [Array{Float64,1}(0) for s in solvers]
times = [Array{Float64,1}(0) for s in solvers]

for i in 1:200
    a = wtedChimera(n,i)
    
    Bt = -dirEdgeVertexMat(a);
    Bt = Bt';
    c1 = rand(size(Bt)[2]);
    u = 0.25*ones(size(Bt)[2]);
    b1 = 0.25*Bt*rand(size(Bt)[2]);

    m = size(Bt)[2]
    n = size(Bt)[1]
    A = [Bt spzeros(n,m); speye(m) speye(m)];
    b = [b1;u];
    
    push!(nlist, n)
    push!(ilist, i)
    push!(edges, nnz(a))
    push!(targetTol, 1e-6)
    
    tmin = Inf
    for si in 1:length(solvers)
        solver = solvers[si]
        
        try
            # we set 20s as the max time per solve
            mysolver = (H -> lapWrapSolver(solver,H,tol=1e-8,maxits=1000,maxtime=20))
            
            tic()
            (x1,y1,s1,numIter) = minCostFlow(Bt,b1,c1,u, lapSolver=mysolver, tol=1e-6, maxtime=(10*tmin));
            push!(times[si], toc())
            
            tmin = min(tmin, times[si][end])
            
            push!(iters[si], numIter)
            push!(precision[si], norm(A * x1 - b) / norm(b))
        catch
            push!(times[si], Inf)
            push!(iters[si], Inf)
            push!(precision[si], Inf)
            
            println("Error for ", i, " and ", solver)
        end
    end
    
    println("Done for ", i)
end

elapsed time: 11.998283828 seconds
elapsed time: 7.315041759 seconds
elapsed time: 7.316607 seconds
Method exceeded time set by user. Stopped after 12 iterations.
elapsed time: 77.857497267 seconds
Done for 1
elapsed time: 7.776100888 seconds
elapsed time: 6.089627398 seconds
elapsed time: 7.769638209 seconds
Method exceeded time set by user. Stopped after 15 iterations.
elapsed time: 67.486531644 seconds
Done for 2


In [6]:
tab = ["nlist"; nlist]
tab = [tab ["ilist"; ilist]]
tab = [tab ["edges"; edges]]
tab = [tab ["targetTol"; targetTol]]

for i in 1:length(solvers)
    name = split(string(solvers[i]), ".")[2]

    
    tab = [tab [name * "_runtime"; times[i]]]
    tab = [tab [name * "_iterations"; iters[i]]]
    tab = [tab [name * "_convergence"; precision[i]]]
end

In [7]:
fn = string("minFlowIPMResults.csv");
writecsv(fn,tab);